# Decoder analysis

Before running this notebook, the `decoder.slurm` jobs needs to be run and the data need to be collated into a single csv file:

`awk 'FNR==1 && NR!=1{next;}{print}' build/*_model.csv > build/decoder_predictions.csv`

In [ ]:
import <- function(pkg) { library(pkg, warn.conflicts=F, quietly=T, character.only=T) }
import("repr")
import("stringr")
import("tidyr")
import("dplyr")
import("ggplot2")
import("lme4")
import("emmeans")

In [ ]:
options(repr.matrix.max.cols=15, repr.matrix.max.rows=20)
my.theme <- theme(legend.text=element_text(size=6),
                  legend.title=element_text(size=6),
                  plot.title = element_text(size=7, hjust=0.5),
                  axis.line=element_line(linewidth=0.25),
                  axis.ticks=element_line(linewidth=0.25),
                  axis.ticks.length=unit(0.05, "cm"),
                  axis.title=element_text(size=7),
                  axis.text=element_text(size=6),
                  strip.placement="outside",
                  strip.text=element_text(size=7),
                  strip.background=element_blank())
no.legend <- theme(legend.position="none")
update_geom_defaults("point", list(fill="white", shape=21, size=0.8))
update_geom_defaults("line", list(linewidth=0.4))

In [ ]:
## Load the results
predictions <- data.table::fread("../build/decoder_predictions.csv")

## Decoder performance - inaudible noise

Figure 7C compares decoder performance on the test motif, averaged across all test motifs, as a function of ensemble size.

In [ ]:
options(repr.plot.width=2.2, repr.plot.height=2.2, repr.plot.res = 450)
p <- (
    predictions
    |> filter(background_dBFS==-100)
    |> group_by(dataset, n_units, seed)
    |> summarize(score=mean(score_actual))
    |> summarize(y=median(score), ymin=quantile(score, 0.25), ymax=quantile(score, 0.75))
    |> ggplot(aes(n_units, y, color=dataset))
    + geom_point(position=position_dodge(width=0.051), size=1.5)
    + geom_linerange(aes(ymin=ymin, ymax=ymax), position=position_dodge(width=0.05))
    + scale_x_log10("Ensemble size")
    + scale_y_continuous("Prediction score (adj R^2)")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/decoder_accuracy.pdf", width=2.1, height=2.1)
print(p)
dev.off()

In [ ]:
## very basic pairwise comparisons, not terribly meaningful because we control the number of replicates
pred_model <- function(df) {
    wilcox.test(score_actual ~ dataset, df)
}

fm <- (
    predictions
    |> filter(background_dBFS==-100)
    |> filter(n_units < 1000)
    |> group_by(n_units)
    |> nest()
    |> transmute(mdl=purrr::map(data, pred_model))
    |> mutate(stats=purrr::map(mdl, broom::tidy))
    |> select(n_units, stats)
    |> unnest(cols=stats)
    |> arrange(n_units)
)
fm

## Noise invariance

In Figure 8, we test for noise invariance by using the decoder trained on clean stimulus to decode
the responses to noisy stimuli. If the brain is filtering out the noise, then the decoded stimuli
should look similar to the foreground.

In [ ]:
options(repr.plot.width=2.2, repr.plot.height=3.5, repr.plot.res = 450)
p <- (
    predictions
    |> filter(n_units %in% c(63, 473, 927))
    |> group_by(n_units, dataset, background_dBFS, seed)
    |> summarize(score=mean(score_actual))
    #|> summarize(y=mean(score), yse=sd(score)/sqrt(n()), ymin=y-yse, ymax=y+yse)
    |> summarize(y=median(score), ymin=quantile(score, 0.25), ymax=quantile(score, 0.75))
    |> ggplot(aes(-30 - background_dBFS, y, color=dataset))
    + facet_grid(rows = vars(n_units))
    + geom_point(size=1.5)
    + geom_linerange(aes(ymin=ymin, ymax=ymax))
    + scale_x_reverse("SNR (dB)")
    + scale_y_continuous("Prediction Score (R^2)")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/decoder_invariance_ensembles.pdf", width=2.2, height=3.5)
print(p)
dev.off()